# Notebook to Test all the functions

## First only the corpus ##

In [1]:
import sys
import pandas as pd
import collections 
import os
import numpy as np
from itertools import chain
from itertools import combinations
sys.path.insert(0, '../..')
from src.experiment_utils.helper_classes import token, span, repository
from src.d02_corpus_statistics.corpus import Corpus
from src.d03_inter_annotator_agreement.inter_annotator_agremment import Inter_Annotator_Agreement, _get_score_article, _get_curation_annotator_score
from definitions import df_annotation_marker
from src.d03_inter_annotator_agreement.inter_annotator_agremment import row_to_span_list, keep_valid_anotations
from src.d03_inter_annotator_agreement.scoring_functions import create_scoring_matrix
from src.d03_inter_annotator_agreement.scoring_functions import f1_exact

from definitions import ROOT_DIR

In [2]:
DEBUG = False
!git branch

  improve_scores
  master
  plots
* update_pygamma


## Test all corpus functions

In [3]:
DEBUG = False
#dataframe_dir = os.path.join(ROOT_DIR,'data/02_processed_to_dataframe', 'preprocessed_dataframe.pkl')
dataframe_dir = '/home/jkuettel/NLP_spark/data/02_processed_to_dataframe/preprocessed_dataframe_test.pkl'

stat_df = pd.read_pickle(dataframe_dir)

In [4]:
stat_df.head()


,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,A,B,C,D,E,F,G
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43,,article 43\r\nexercise of the delegation\r\n1....,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR0 annotator:Curation layer:Instrum...,[span id:A1 annotator:A layer:Instrumenttypes ...,[span id:B1 annotator:B layer:Policydesignchar...,,,,,
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nreal-world co2 emissions and fue...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[C, D]",[span id:CUR36 annotator:Curation layer:Instru...,,,[span id:C1 annotator:C layer:Instrumenttypes ...,[span id:D1 annotator:D layer:Policydesignchar...,,,
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11,,article 11\r\njoint projects between member st...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C]",[span id:CUR116 annotator:Curation layer:Instr...,,[span id:B28 annotator:B layer:Instrumenttypes...,[span id:C58 annotator:C layer:Instrumenttypes...,,,,
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56,,article 56\r\namendments to directive (eu) 201...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[A, B]",[span id:CUR202 annotator:Curation layer:Polic...,[span id:A38 annotator:A layer:Policydesigncha...,[span id:B129 annotator:B layer:Policydesignch...,,,,,
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_03,,article 3\r\nbinding overall union target for ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[B, C, D]",[span id:CUR211 annotator:Curation layer:Instr...,,[span id:B138 annotator:B layer:Instrumenttype...,[span id:C165 annotator:C layer:Instrumenttype...,[span id:D27 annotator:D layer:Instrumenttypes...,,,


In [5]:
test_evaluator = Inter_Annotator_Agreement(stat_df, front_and_whereas = False, DEBUG = DEBUG)
test_evaluator.keep_only_finished_articles()
print(test_evaluator.df.shape)

(412, 13)


In [6]:
scoring_metrics = ['f1_exact_pygamma', 'f1_tokenwise_pygamma', 'f1_partial_pygamma', 'f1_heuristic', 'f1_exact', 'f1_tokenwise', 'f1_positional']

In [7]:

test_evaluator.append_total_score_per_article(scoring_metrics)
test_evaluator.df.head()

  0%|          | 1/412 [00:00<00:01, 391.81it/s]


TypeError: CombinedCategoricalDissimilarity.__init__() got an unexpected keyword argument 'categories'

In [9]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_exact_pygamma_score': 0.40028724799007476,
 'f1_tokenwise_pygamma_score': 0.397687590370067,
 'f1_partial_pygamma_score': 0.5159440267362017,
 'f1_heuristic_score': 0.5275951402554365,
 'f1_exact_score': 0.40028724799007476,
 'f1_tokenwise_score': 0.5210605573521196,
 'f1_positional_score': 0.7168747609545078}

In [10]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_exact_pygamma_score': 0.40028724799007476,
 'f1_tokenwise_pygamma_score': 0.397687590370067,
 'f1_partial_pygamma_score': 0.5159440267362017,
 'f1_heuristic_score': 0.5275951402554365,
 'f1_exact_score': 0.40028724799007476,
 'f1_tokenwise_score': 0.5210605573521196,
 'f1_positional_score': 0.7168747609545078}

In [ ]:
#test_evaluator.get_total_score_df(weight_by = 'no_weighting')

In [9]:
(0.5439488370573863 + 0.4981722776468529)/2

0.5210605573521196

In [8]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_exact_score': 0.40028724799007476,
 'f1_tokenwise_score': 0.397687590370067,
 'f1_partial_score': 0.5159440267362017,
 'f1_heuristic_score': 0.5273362748620107,
 'f1_exact_brute_force_score': 0.40028724799007476,
 'f1_article_tokenwise_score': 0.4981722776468529,
 'f1_article_tokenwise_r_score': 0.5439488370573863,
 'f1_positional_article_tokenwise_score': 0.7168747609545078}

In [8]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_exact_score': 0.40028724799007476,
 'f1_tokenwise_score': 0.397687590370067,
 'f1_partial_score': 0.5159440267362017,
 'f1_heuristic_score': 0.5273362748620107,
 'f1_exact_brute_force_score': 0.40028724799007476,
 'f1_article_tokenwise_score': 0.24908613882342645,
 'f1_article_tokenwise_r_score': 0.5439488370573863,
 'f1_positional_article_tokenwise_score': 0.7168747609545078}

In [10]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_exact_score': 0.40028724799007476,
 'f1_tokenwise_score': 0.397687590370067,
 'f1_partial_score': 0.5159440267362017,
 'f1_heuristic_score': 0.5273362748620107,
 'f1_exact_brute_force_score': 0.40028724799007476,
 'f1_article_tokenwise_score': 0.750468016406623,
 'f1_article_tokenwise_r_score': 0.772280035777206,
 'f1_positional_article_tokenwise_score': 0.7168747609545078}

In [10]:
for i in range(0,2):
    print(i)

0
1


In [9]:
test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['D'][0].tag_ in test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['C'][0]

TypeError: argument of type 'span' is not iterable

In [9]:
test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['C'][0].tokens[0].get_token_tags(annotators = 'annotators')

['RegulatoryInstr', 'RegulatoryInstr']

In [ ]:
ann_span.tag_ in cur_tok.get_token_spans(annotators = 'annotators')

In [ ]:
test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Tokens'][8].get_token_spans(annotators = 'annotators')

In [ ]:
 test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Tokens'][8].get_token_spans(annotators = 'annotators')

In [34]:
from collections import Counter
a =  [3,6]
b = [3,6]

a_l = []

b_l = []


for i in a:
    a_l.append(test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['D'][i])
    
for i in b:
    b_l.append(test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['C'][i])
    
a_l_t = list(chain.from_iterable([s.tokens for s in a_l]))
b_l_t = list(chain.from_iterable([s.tokens for s in b_l]))

print(a_l)
print()
print(b_l)
print()
print()

print(a_l_t)
print()
print(b_l_t)


[span id:D118 annotator:D layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:247 stop:259 text:requirements, span id:D121 annotator:D layer:Technologyandapplicationspecificity type:EnergySpecificity tag:Energy_LowCarbon start:123 stop:131 text:hydrogen]

[span id:C252 annotator:C layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:247 stop:259 text:requirements, span id:C255 annotator:C layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:172 stop:191 text:hydrogen components]


[start:247 stop:259 text:requirements tag_count:1, start:123 stop:131 text:hydrogen tag_count:2]

[start:247 stop:259 text:requirements tag_count:1, start:172 stop:180 text:hydrogen tag_count:2, start:181 stop:191 text:components tag_count:1]


In [38]:
    
list((Counter(a_l_t) & Counter(b_l_t)).elements())

[start:247 stop:259 text:requirements tag_count:1]

In [40]:
test = dict(Counter(a_l_t) & Counter(b_l_t))

In [42]:
for t in test:
    print(t.spans)
    print()

[span id:D118 annotator:D layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:247 stop:259 text:requirements, span id:C252 annotator:C layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:247 stop:259 text:requirements, span id:CUR368 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:247 stop:259 text:requirements]



In [43]:
len((Counter(a_l) & Counter(b_l)).elements())

TypeError: object of type 'itertools.chain' has no len()

In [14]:
from collections import Counter

tp = list((Counter(a_l) & Counter(b_l)).elements())



NameError: name 'a_l' is not defined

In [18]:
test = [test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Curation'][4], test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Curation'][4]]
list(chain.from_iterable([l.tokens for l in test]))


[start:93 stop:98 text:motor tag_count:1,
 start:99 stop:107 text:vehicles tag_count:1,
 start:93 stop:98 text:motor tag_count:1,
 start:99 stop:107 text:vehicles tag_count:1]

In [19]:
test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Tokens'][8].spans

[span id:D115 annotator:D layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:55 stop:67 text:requirements,
 span id:C249 annotator:C layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:55 stop:67 text:requirements,
 span id:CUR365 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:55 stop:67 text:requirements]

In [6]:
span_tokens = test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Tokens']
sp = test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Curation'][1]

map(lambda tok: tok.add_span(sp), span_tokens)

In [7]:
span_tokens

[start:0 stop:7 text:article tag_count:0,
 start:7 stop:8 text:  tag_count:0,
 start:8 stop:9 text:1 tag_count:0,
 start:11 stop:18 text:subject tag_count:0,
 start:19 stop:25 text:matter tag_count:0,
 start:27 stop:31 text:this tag_count:0,
 start:32 stop:42 text:regulation tag_count:0,
 start:43 stop:54 text:establishes tag_count:0,
 start:55 stop:67 text:requirements tag_count:0,
 start:68 stop:71 text:for tag_count:0,
 start:72 stop:75 text:the tag_count:0,
 start:76 stop:89 text:type-approval tag_count:0,
 start:90 stop:92 text:of tag_count:0,
 start:93 stop:98 text:motor tag_count:0,
 start:99 stop:107 text:vehicles tag_count:0,
 start:108 stop:112 text:with tag_count:0,
 start:113 stop:119 text:regard tag_count:0,
 start:120 stop:122 text:to tag_count:0,
 start:123 stop:131 text:hydrogen tag_count:0,
 start:132 stop:142 text:propulsion tag_count:0,
 start:143 stop:146 text:and tag_count:0,
 start:147 stop:150 text:for tag_count:0,
 start:151 stop:154 text:the tag_count:0,
 start

In [20]:
span_tokens = test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Tokens']
sp = test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Curation'][1]
sp

span id:CUR366 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:76 stop:89 text:type-approval

In [21]:
span_id = 'CUR1'
[tok.add_span(sp) for tok in span_tokens ]
span_tokens[0].spans

[span id:CUR366 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:76 stop:89 text:type-approval]

In [9]:
span_tokens

[start:0 stop:7 text:article tag_count:1,
 start:7 stop:8 text:  tag_count:1,
 start:8 stop:9 text:1 tag_count:1,
 start:11 stop:18 text:subject tag_count:1,
 start:19 stop:25 text:matter tag_count:1,
 start:27 stop:31 text:this tag_count:1,
 start:32 stop:42 text:regulation tag_count:1,
 start:43 stop:54 text:establishes tag_count:1,
 start:55 stop:67 text:requirements tag_count:1,
 start:68 stop:71 text:for tag_count:1,
 start:72 stop:75 text:the tag_count:1,
 start:76 stop:89 text:type-approval tag_count:1,
 start:90 stop:92 text:of tag_count:1,
 start:93 stop:98 text:motor tag_count:1,
 start:99 stop:107 text:vehicles tag_count:1,
 start:108 stop:112 text:with tag_count:1,
 start:113 stop:119 text:regard tag_count:1,
 start:120 stop:122 text:to tag_count:1,
 start:123 stop:131 text:hydrogen tag_count:1,
 start:132 stop:142 text:propulsion tag_count:1,
 start:143 stop:146 text:and tag_count:1,
 start:147 stop:150 text:for tag_count:1,
 start:151 stop:154 text:the tag_count:1,
 start

In [21]:
[span_ for span_  in span_tokens[1].spans if 'Curation' in span_.annotator]


[span id:CUR366 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:76 stop:89 text:type-approval]

In [8]:
span_tokens[0].get_token_spans(annotators = 'Curation')

[span id:CUR366 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:76 stop:89 text:type-approval]

In [8]:
test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Tokens'][8]test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Fabian']

[span id:D115 annotator:Fabian layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:55 stop:67 text:requirements,
 span id:D116 annotator:Fabian layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:76 stop:89 text:type-approval,
 span id:D117 annotator:Fabian layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:155 stop:168 text:type-approval,
 span id:D118 annotator:Fabian layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:247 stop:259 text:requirements,
 span id:D119 annotator:Fabian layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_Other start:93 stop:107 text:motor vehicles,
 span id:D120 annotator:Fabian layer:Technologyandapplicationspecificity type:ApplicationSpecificity tag:App_LowCarbon start:123 stop:142 text:hydrogen propulsion,
 span id:D121 annotator:Fabian layer:Technologyandapplicationspecificity type:EnergySpecificity tag:Energy_LowCarbon start:123 stop:131 text:hydrogen,
 span id:

In [9]:
test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Onerva']

[span id:C249 annotator:Onerva layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:55 stop:67 text:requirements,
 span id:C250 annotator:Onerva layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:76 stop:89 text:type-approval,
 span id:C251 annotator:Onerva layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:155 stop:168 text:type-approval,
 span id:C252 annotator:Onerva layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:247 stop:259 text:requirements,
 span id:C253 annotator:Onerva layer:Technologyandapplicationspecificity type:ApplicationSpecificity tag:App_Other start:93 stop:107 text:motor vehicles,
 span id:C254 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:123 stop:142 text:hydrogen propulsion,
 span id:C255 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:172 stop:191 text:hydrogen componen

In [9]:
test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01']['Curation']

[span id:CUR365 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:55 stop:67 text:requirements,
 span id:CUR366 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:76 stop:89 text:type-approval,
 span id:CUR367 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:155 stop:168 text:type-approval,
 span id:CUR368 annotator:Curation layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:247 stop:259 text:requirements,
 span id:CUR369 annotator:Curation layer:Technologyandapplicationspecificity type:ApplicationSpecificity tag:App_Other start:93 stop:107 text:motor vehicles,
 span id:CUR370 annotator:Curation layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:123 stop:142 text:hydrogen propulsion,
 span id:CUR371 annotator:Curation layer:Technologyandapplicationspecificity type:EnergySpecificity tag:Energy_LowCarbon start:123 stop:

In [26]:
test_evaluator.df.loc['EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01'][13:]

f1_exact_score        0.545455
f1_tokenwise_score    0.428571
f1_partial_score      0.545455
f1_heuristic_score    0.545455
Name: EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01, dtype: object

In [8]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_exact_score': 0.40028724799007476,
 'f1_tokenwise_score': 0.397687590370067,
 'f1_partial_score': 0.5159440267362017,
 'f1_heuristic_score': 0.5273362748620107,
 'f1_exact_brute_force_score': 0.40028724799007476,
 'f1_article_tokenwise_score': 0.0,
 'f1_article_tokenwise_r_score': 0.0}

In [21]:
from src.d03_inter_annotator_agreement.span_matching import create_tuples_pygamma

rep = repository.from_repository_name('EU_32009R0079_Title_0_Chapter_0_Section_0_Article_01')
spanlist = test_evaluator.get_span_list(conditional_rep = rep, annotators = 'annotators')
for tu in create_tuples_pygamma(spanlist):
    for i in tu:
        print(i)
    print()

span id:D115 annotator:Fabian layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:55 stop:67 text:requirements
span id:C249 annotator:Onerva layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:55 stop:67 text:requirements

span id:D116 annotator:Fabian layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:76 stop:89 text:type-approval
span id:C250 annotator:Onerva layer:Instrumenttypes type:InstrumentType tag:RegulatoryInstr start:76 stop:89 text:type-approval

span id:D119 annotator:Fabian layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_Other start:93 stop:107 text:motor vehicles
span id:C253 annotator:Onerva layer:Technologyandapplicationspecificity type:ApplicationSpecificity tag:App_Other start:93 stop:107 text:motor vehicles

span id:D121 annotator:Fabian layer:Technologyandapplicationspecificity type:EnergySpecificity tag:Energy_LowCarbon start:123 stop:131 text:hydrogen
span id:None annotator:Onerva laye

In [ ]:
 collections.Counter(self.tokens) & collections.Counter(other.tokens)
            num_same = sum(common.values())

            if num_same == 0:
                return 0

            precision = 1.0 * num_same / len(self.tokens)
            recall = 1.0 * num_same / len(other.tokens)
            f1 = (2*precision * recall) / (precision + recall)
            return f1
        else:
            raise ValueError('None of the mentioned')

In [ ]:
tu_list = create_tuples_pygamma(spanlist)

In [ ]:
f1_tokenwise(create_tuples_pygamma(spanlist))

In [ ]:
len([n_tuple[0] for n_tuple in tu_list if n_tuple[0].tag_ != None ])

In [ ]:
for i, row in test_evaluator.df.iterrows():
    print(row.name)
    print(len(row['Curation']))

### all the non curated articles

In [ ]:
test_evaluator.df_non_curated.head()

### Chech if there are finished articles where there are less than 2 finsihed annotators

In [ ]:
test_evaluator.df[test_evaluator.df.apply(lambda x: len(x['Finished_Annotators'])  < 2,axis=1)]

## Test small helpfer functions

### row to spanlist

In [ ]:
row_to_span_list(test_evaluator.df.loc['EU_32009R0397_Title_0_Chapter_0_Section_0_Article_02'])

In [ ]:
repo = repository.from_repository_name('EU_32009R0397_Title_0_Chapter_0_Section_0_Article_02')
test_evaluator.get_span_list(repo, columns = 'annotators')

### _get_score_article

** TO DO: TEST SCORE OF A FEW KNOWN SCORES **

## Observe nan scores

In [ ]:

test_evaluator.df[test_evaluator.df['f1_heuristic_score'].isnull()]

In [ ]:
repo = repository.from_repository_name('EU_32009R0079_Title_0_Chapter_0_Section_0_Article_14')

In [ ]:
taglist = test_evaluator.get_span_list(repo, columns = 'annotators')

In [ ]:
taglist

In [ ]:
test_evaluator.get_score_spanlist(taglist, scoring_metric = 'f1_heuristic')

## Test weighting of total score

### no weighting

In [10]:
for scoring_metric in scoring_metrics:
    print(scoring_metric, test_evaluator.df[scoring_metric].mean())

KeyError: 'f1_exact'

In [ ]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

### manually weight by tokens

In [ ]:
for scoring_metric in scoring_metrics:
    print(scoring_metric, (test_evaluator.df.Tokens.map(len) * test_evaluator.df[scoring_metric + '_score']).sum()/test_evaluator.df.Tokens.map(len).sum())

In [ ]:
test_evaluator.get_total_score_df(weight_by = 'Tokens')

In [ ]:
test_evaluator.df.head()

### manually weight by spans

In [ ]:
from src.d03_inter_annotator_agreement.inter_annotator_agremment import _get_span_count_in_row
for scoring_metric in scoring_metrics:

    test_evaluator.df['Span_len'] = test_evaluator.df.apply(lambda row: _get_span_count_in_row(row, cols = row['Finished_Annotators']), axis = 1)
    print((test_evaluator.df['Span_len'] * test_evaluator.df[scoring_metric + '_score']).sum()/test_evaluator.df['Span_len'].sum())


In [ ]:
test_evaluator.get_total_score_df(weight_by = 'Spans')

## Test individual score

### Test if individual score averaged yields total score

In [ ]:

for sco in scoring_metrics:
    tokens = 0
    res = 0

    sco = sco + '_score'
    for ann in test_evaluator.annotators:
        tok = len(list(chain.from_iterable(test_evaluator.df[test_evaluator.df.apply(lambda x: ann in x['Finished_Annotators'],axis=1)]['Tokens'])))

        res += test_evaluator.get_total_score_df(annotator = ann)[sco] * tok
        tokens += tok
    print(sco, res / tokens)

# Testing get_score_spanlist

In [ ]:

test_dir = repository.from_repository_name('EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43')
span_list = test_evaluator.get_span_list(test_dir, ['Alisha', 'Fride'])
len(span_list)

In [ ]:
test_evaluator.get_score_spanlist(span_list, 'f1_exact')

In [ ]:
test_dir = repository.from_repository_name('EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56')
span_list1 = test_evaluator.get_span_list(test_dir, ['Alisha', 'Fride'])
len(span_list1)

In [ ]:
test_evaluator.get_score_spanlist(span_list1, 'f1_exact')

In [ ]:
span_list2 = span_list + span_list1
test_evaluator.get_score_spanlist(span_list2, 'f1_exact')

In [ ]:
(test_evaluator.get_score_spanlist(span_list, 'f1_exact') * len(span_list) + test_evaluator.get_score_spanlist(span_list1, 'f1_exact') * len(span_list1)) / (len(span_list) + len(span_list1))

## Test if this is also true for a long spanlist of multiple repositorys

In [ ]:
#itnitialize test evaluator again, this time with debug 
test_evaluator_debug = Inter_Annotator_Agreement(stat_df, DEBUG = True)
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic']

test_evaluator_debug.append_total_score_per_article(scoring_metrics)
test_evaluator_debug.append_score_to_curation(scoring_metrics)

In [ ]:
test_evaluator_debug.df.head()

In [ ]:
test_dir = repository()

In [ ]:
#get all tags of fabian and curation
span_list_Fabian = test_evaluator_debug.get_span_list(test_dir, ['Curation', 'Fabian'])
set([spn.rep for spn in span_list_Fabian if spn.annotator == 'Fabian'])

In [ ]:
#get all tags of Fride and curation
span_list_fride = test_evaluator_debug.get_span_list(test_dir, ['Curation', 'Fride'])
set([spn.rep for spn in span_list_fride if spn.annotator == 'Fride'])

In [ ]:
weigh_by = 'Spans'

In [ ]:
#inter annotator agreement to curation

test_evaluator_debug.get_to_curation_score(weight_by = weigh_by)['Fabian']

In [ ]:
for sco in scoring_metrics:
    print(sco, test_evaluator_debug.get_score_spanlist(test_dir, columns = ['Curation', 'Fabian'], scoring_metric = sco, weight_by = weigh_by))

In [ ]:
test_evaluator_debug.get_to_curation_score(weight_by = weigh_by)['Fride']

In [ ]:
for sco in scoring_metrics:
    print(sco, test_evaluator_debug.get_score_spanlist(test_dir, columns = ['Curation', 'Fride'], scoring_metric = sco, weight_by = weigh_by))

### Manually weight Fride by Spans

In [ ]:
#should be the same as above
Fride_df = test_evaluator_debug.df[test_evaluator_debug.df.apply(lambda x : len(x['Fride']) >=2 and len(x['Curation']) >=2,axis=1)]
scoring_metric = 'f1_partial'
score = 0
total_count = 0

for i, row in Fride_df.iterrows():
    span_list = list(chain.from_iterable(row[['Fride', 'Curation']]))
    score_article = _get_score_article(span_list, scoring_metric, finished_annotators = ['Fride', 'Curation'])
    score += score_article * len(span_list)
    total_count += len(span_list)
print(score/total_count)


### Manually weight Fabian by Spans

In [ ]:
#should be the same as above
Fride_df = test_evaluator_debug.df[test_evaluator_debug.df.apply(lambda x : len(x['Fabian']) >=2 and len(x['Curation']) >=2,axis=1)]
scoring_metric = 'f1_partial'
score = 0
total_count = 0

for i, row in Fride_df.iterrows():
    span_list = list(chain.from_iterable(row[['Fabian', 'Curation']]))
    score_article = _get_score_article(span_list, scoring_metric, finished_annotators = ['Fabian', 'Curation'])
    score += score_article * len(span_list)
    total_count += len(span_list)
print(score/total_count)

## Test get_score_anotator

In [ ]:
test_evaluator_debug.get_total_score_df(annotator = 'Fride', weight_by = 'Tokens')

In [ ]:

annotator_df = test_evaluator_debug.df[test_evaluator_debug.df.apply(lambda x: 'Fride' in x['Finished_Annotators'],axis=1)]

### f1 partial

In [ ]:
#should be the same as this one
sum = 0
for i, row in annotator_df.iterrows():
    sum += len(row['Tokens']) * row['f1_partial_score']
sum/len(list(chain.from_iterable(annotator_df['Tokens'])))

### f1 tokenwise

In [ ]:
sum = 0
for i, row in annotator_df.iterrows():
    sum += len(row['Tokens']) * row['f1_tokenwise_score']
sum/len(list(chain.from_iterable(annotator_df['Tokens'])))

test_evaluator_debug.df## Test heuristic_f1

In [ ]:
test_evaluator_debug.df.head()

In [ ]:
index = 3
test_dir = test_evaluator_debug.df['Curation'][index][1].rep
span_list = test_evaluator_debug.get_span_list(test_dir, columns = 'annotators')
from src.d03_inter_annotator_agreement.scoring_functions import f1_heuristic
test_dir

In [ ]:
[spn for spn in span_list if spn.annotator == 'Alisha']

In [ ]:
[spn for spn in span_list if spn.annotator == 'Fride']

In [ ]:
f1_heuristic(span_list,( 'Alisha', 'Fride'))

In [ ]:
### Evaluating by hand:
98/133

## Test Pygamma 

In [ ]:
# create custom scoring matrix
category_list, cat_dissimilarity_matrix = create_scoring_matrix(os.path.join(ROOT_DIR,'src/experiment_utils/tag_set.json'),  soft_tagset_dissimilarity = True)
cat_dissimilarity_matrix_test = np.ones((len(category_list), len(category_list)))
np.fill_diagonal(cat_dissimilarity_matrix_test,0)

In [ ]:
#compare influence of soft dissimilarity
#without matrix
test_evaluator = Inter_Annotator_Agreement(stat_df)
test_evaluator.keep_only_finished_articles()
scoring_metrics = ['pygamma']
test_evaluator.append_total_score_per_article(scoring_metrics)
test_evaluator.get_total_score_df(weight_by = 'no_weighting')


In [ ]:
#with matrix
test_evaluator = Inter_Annotator_Agreement(stat_df)
test_evaluator.keep_only_finished_articles()
scoring_metrics = ['pygamma']
test_evaluator.append_total_score_per_article(scoring_metrics, category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix)
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

In [ ]:
#with test matrix
test_evaluator = Inter_Annotator_Agreement(stat_df)
test_evaluator.keep_only_finished_articles()
scoring_metrics = ['pygamma']
test_evaluator.append_total_score_per_article(scoring_metrics, category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix_test)
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

In [ ]:
#with test matrix
test_evaluator = Inter_Annotator_Agreement(stat_df, front_and_whereas = False, DEBUG = False)
test_evaluator.keep_only_finished_articles()
scoring_metrics = ['pygamma']
test_evaluator.append_total_score_per_article_parallel(scoring_metrics, category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix_test)
test_evaluator.get_total_score_df(weight_by = 'no_weighting')


### Curation

In [ ]:
#compare influence of soft dissimilarity
#without matrix
test_evaluator = Inter_Annotator_Agreement(stat_df)
test_evaluator.keep_only_finished_articles()
scoring_metrics = ['pygamma']
test_evaluator.append_score_to_curation(scoring_metrics)
test_evaluator.get_to_curation_score(weight_by = 'no_weighting')

In [ ]:
#compare influence of soft dissimilarity
#without matrix
test_evaluator = Inter_Annotator_Agreement(stat_df)
test_evaluator.keep_only_finished_articles()
scoring_metrics = ['pygamma']
test_evaluator.append_score_to_curation(scoring_metrics, category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix)
test_evaluator.get_to_curation_score(weight_by = 'no_weighting')

In [ ]:
#compare influence of soft dissimilarity
#without matrix
test_evaluator = Inter_Annotator_Agreement(stat_df)
test_evaluator.keep_only_finished_articles()
scoring_metrics = ['pygamma']
test_evaluator.append_score_to_curation(scoring_metrics, category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix_test)
test_evaluator.get_to_curation_score(weight_by = 'no_weighting')

In [ ]:
np.arange(3)

# Test total

In [ ]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic']
scoring_metrics.sort()
test_evaluator.append_total_score_per_article(scoring_metrics)

In [ ]:
category_list, cat_dissimilarity_matrix = create_scoring_matrix(os.path.join(ROOT_DIR,'src/experiment_utils/tag_set.json'),  soft_tagset_dissimilarity = True, soft_layer_dissimilarity = False)
test_evaluator.append_total_score_per_article(scoring_metrics = ['pygamma'], category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix)

## To Curation

In [ ]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic']
category_list, cat_dissimilarity_matrix = create_scoring_matrix(os.path.join(ROOT_DIR,'src/experiment_utils/tag_set.json'),  soft_tagset_dissimilarity = True, soft_layer_dissimilarity = False)
test_evaluator.append_score_to_curation(scoring_metrics = scoring_metrics, category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix)

#test_evaluator.append_score_to_curation(scoring_metrics = scoring_metrics)



In [ ]:
test_evaluator.df.head()

## Test difference in pygamma matching algo for f1


In [ ]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial']
category_list, cat_dissimilarity_matrix = create_scoring_matrix(os.path.join(ROOT_DIR,'src/experiment_utils/tag_set.json'),  soft_tagset_dissimilarity = True, soft_layer_dissimilarity = False)


In [ ]:
# first add no dissimilary matrix for the pygamma matching algo

test_evaluator_no_matrix = Inter_Annotator_Agreement(stat_df, front_and_whereas = False, DEBUG = DEBUG)
test_evaluator_no_matrix.keep_only_finished_articles()

test_evaluator_no_matrix.append_total_score_per_article(scoring_metrics)
test_evaluator_no_matrix.append_score_to_curation(scoring_metrics)

print('Total score:')
print(test_evaluator_no_matrix.get_total_score_df(weight_by = 'no_weighting'))
print('Curation score:')      
print(test_evaluator_no_matrix.get_to_curation_score(weight_by = 'no_weighting'))

In [ ]:
# now add the categories for the pygamma matching algo
test_evaluator_scoring_matrix = Inter_Annotator_Agreement(stat_df, front_and_whereas = False, DEBUG = DEBUG)
test_evaluator_scoring_matrix.keep_only_finished_articles()

test_evaluator_scoring_matrix.append_total_score_per_article(scoring_metrics, category_list_matching = category_list, cat_dissimilarity_matrix_matching = cat_dissimilarity_matrix)
test_evaluator_scoring_matrix.append_score_to_curation(scoring_metrics, category_list_matching = category_list, cat_dissimilarity_matrix_matching = cat_dissimilarity_matrix)

print('Total score:')
print(test_evaluator_scoring_matrix.get_total_score_df(weight_by = 'no_weighting'))
print('Curation score:')      
print(test_evaluator_scoring_matrix.get_to_curation_score(weight_by = 'no_weighting'))

**The scores are very simmilar, feeding no dissimilarity matrix yields a higher score!**

## Check if get score spanlist yields the same


In [ ]:
weight_by = 'no_weighting'
scoring_metric = 'f1_exact'
roud_to = 3


In [ ]:
sorted(scoring_metrics)

In [ ]:
row_number = 0
for i, col in test_evaluator.df.iterrows():
    if (row_number % 20 == 0):
        print(row_number)
    
    name= col.name
    test_rep = repository.from_repository_name(name)
    finished_annotators = test_evaluator.df.loc[name]['Finished_Annotators']

    test_span_list_O = test_evaluator.get_span_list(test_rep, columns = ['Curation', finished_annotators[0]])
    test_span_list_1 = test_evaluator.get_span_list(test_rep, columns = ['Curation', finished_annotators[1]])
    
    if len(list(set([spn.annotator for spn in test_span_list_O]))) < 2 or len(list(set([spn.annotator for spn in test_span_list_1]))) < 2:
        continue

    score_df_0 = test_evaluator.df.loc[name][(finished_annotators[0] + '_to_curation')]
    score_df_1 = test_evaluator.df.loc[name][(finished_annotators[1] + '_to_curation')]

    score_array_0 = np.zeros(len(scoring_metrics))
    score_array_1 = np.zeros(len(scoring_metrics))

        
    for i, sco in enumerate(sorted(scoring_metrics)):

        score_array_0[i] = test_evaluator.get_score_spanlist(test_span_list_O, weight_by = weight_by, scoring_metric = sco)
        score_array_1[i] = test_evaluator.get_score_spanlist(test_span_list_1, weight_by = weight_by, scoring_metric = sco)
        
    assert (np.round(score_df_0,roud_to) == np.round(score_array_0,roud_to)).all(), f" score of {col} of annotator {finished_annotators[0]}: df score {np.round(score_df_0,roud_to)} spanlist score {np.round(score_array_0,roud_to)}"
    assert (np.round(score_df_1,roud_to) == np.round(score_array_1,roud_to)).all(), f" score of {col} of annotator {finished_annotators[1]}: df score {np.round(score_df_1,roud_to)} spanlist score {np.round(score_array_1,roud_to)}"
    row_number +=1

## Investigate differences

In [ ]:
name = 'EU_32006L0066_Title_0_Chapter_0_Section_0_Article_11'
test_rep = repository.from_repository_name(name)
finished_annotators = test_evaluator.df.loc[name]['Finished_Annotators']

test_span_list_O = test_evaluator.get_span_list(test_rep, columns = ['Curation', finished_annotators[0]])
test_span_list_1 = test_evaluator.get_span_list(test_rep, columns = ['Curation', finished_annotators[1]])

score_df_0 = test_evaluator.df.loc[name][(finished_annotators[0] + '_to_curation')]
score_df_1 = test_evaluator.df.loc[name][(finished_annotators[1] + '_to_curation')]



In [ ]:
print(sorted(scoring_metrics))
finished_annotators

In [ ]:
score_df_1

In [ ]:
for sco in sorted(scoring_metrics):
    print(test_evaluator.get_score_spanlist(test_span_list_1, weight_by = 'no_weighting', scoring_metric = sco))
    

In [ ]:
for sco in sorted(scoring_metrics):
    print(test_evaluator._get_score_(test_span_list_1, weight_by = 'no_weighting', scoring_metric = sco))

In [ ]:
_get_score_article(test_span_list_1, 'f1_partial', ['Curation', finished_annotators[1]], category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix)

In [ ]:
_get_curation_annotator_score(test_evaluator.df.loc[name], sorted(scoring_metrics), sorted(scoring_metrics), finished_annotators[1])

In [ ]:
test_evaluator_11 = Inter_Annotator_Agreement(stat_df.loc[[name]], front_and_whereas = False)
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic']
category_list, cat_dissimilarity_matrix = create_scoring_matrix(os.path.join(ROOT_DIR,'src/experiment_utils/tag_set.json'),  soft_tagset_dissimilarity = True, soft_layer_dissimilarity = False)
test_evaluator_11.append_score_to_curation(scoring_metrics = scoring_metrics, category_list = category_list, cat_dissimilarity_matrix = cat_dissimilarity_matrix)


In [ ]:
test_evaluator_11.df.loc[name]['Onerva_to_curation']

## Individually

### Check if adding multiple scores yields the same as adding only a single score

In [ ]:
scoring_metrics = ['f1_exact', 'f1_tokenwise','f1_partial', 'f1_heuristic']

weighting_methods = ['no_weighting', 'Spans', 'Tokens']

annotators = ['Fabian', 'Onerva', 'Fride', 'Alisha']
tot_results = test_evaluator.append_score_to_curation(scoring_metrics)

In [ ]:


for wei_meth in weighting_methods:
    print(f"  \n \n elaborating {wei_meth} ... \n \n")
    tot_results = test_evaluator.get_to_curation_score(weight_by = wei_meth)
    
    for sco_meth in scoring_metrics:
        print(f" elaborating {sco_meth} ...")
        test_evaluator_ind = Inter_Annotator_Agreement(stat_df, DEBUG = DEBUG)
        test_evaluator_ind.keep_only_finished_articles()
        sco_meths = [sco_meth]
        test_evaluator_ind.append_score_to_curation(sco_meths)
        ind_results = test_evaluator_ind.get_to_curation_score(weight_by = wei_meth)
        
        for ann in annotators:
            assert np.round(tot_results[ann][sco_meth],3) == np.round(ind_results[ann][sco_meth],3), f"tot results of annotator {ann} was {tot_results[ann][sco_meth]} and ind result was {ind_results[ann][sco_meth]}"
        
        
    

### Check if this also works when adding the scores separately

In [ ]:
scoring_metrics = ['f1_exact', 'f1_tokenwise']

weighting_methods = ['no_weighting', 'Spans', 'Tokens']

annotators = ['Fabian', 'Onerva', 'Fride', 'Alisha']
tot_results = test_evaluator.append_score_to_curation(scoring_metrics)
scoring_metrics = ['f1_partial', 'f1_heuristic']
tot_results = test_evaluator.append_score_to_curation(scoring_metrics)

scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic']

In [ ]:
for wei_meth in weighting_methods:
    print(f"  \n \n elaborating {wei_meth} ... \n \n")
    tot_results = test_evaluator.get_to_curation_score(weight_by = wei_meth)
    
    for sco_meth in scoring_metrics:
        print(f" elaborating {sco_meth} ...")
        test_evaluator_ind = Inter_Annotator_Agreement(stat_df, DEBUG = DEBUG)
        test_evaluator_ind.keep_only_finished_articles()
        sco_meths = [sco_meth]
        test_evaluator_ind.append_score_to_curation(sco_meths)
        ind_results = test_evaluator_ind.get_to_curation_score(weight_by = wei_meth)
        
        for ann in annotators:
            assert np.round(tot_results[ann][sco_meth],3) == np.round(ind_results[ann][sco_meth],3), f"tot results of annotator {ann} was {tot_results[ann][sco_meth]} and ind result was {ind_results[ann][sco_meth]}"
        
        
        
        


In [ ]:
results_tot_no

In [ ]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic']
for sco_meth in scoring_metrics:
    print(f" elaborating {sco_meth} ...")
    test_evaluator_ind = Inter_Annotator_Agreement(stat_df, DEBUG = DEBUG)
    test_evaluator_ind.keep_only_finished_articles()
    scoring_metrics = [sco_meth]
    test_evaluator_ind.append_score_to_curation(scoring_metrics)
    ind_results = test_evaluator_ind.get_to_curation_score(weight_by = 'no_weighting')
    print(ind_results)

In [ ]:
test= [False, False, False]
max(test)

## Test parallel implementation


In [ ]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic', 'pygamma']
test_evaluator = Inter_Annotator_Agreement(stat_df, front_and_whereas = False)
test_evaluator.append_total_score_per_article(scoring_metrics)

In [ ]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic', 'pygamma']
test_evaluator_parallel = Inter_Annotator_Agreement(stat_df, front_and_whereas = False)
test_evaluator_parallel.append_total_score_per_article(scoring_metrics, parallel = True)

In [ ]:
test_evaluator.get_total_score()